# Importing librairies

In [1]:
# Used to manipule and analyse data
import pandas as pd

# Used for our task for cleanning
import io
import re
import string
import tokenize
import warnings
import mdtex2html
import numpy as np
from langdetect import detect, DetectorFactory
from bs4 import BeautifulSoup
from cleantext import clean
from io import StringIO

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


Path to data

In [2]:
DATA_PATH = "../data/"

Features needed for markdown's cleaning

In [3]:
# URL's remover
RMV_URL = re.compile(r'https?://\S+|www\.\S+')
# Equation's remover
RMV_EQU = r"(\$+)(?:(?!\1)[\s\S])*\1"
# Emojis detector
EMOJI_PATTERN = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
DetectorFactory.seed = 0
warnings.filterwarnings('ignore')

Create new dataframe 

In [4]:
new_df = pd.DataFrame(columns=['cell_type','source','title'])
df = pd.read_csv(DATA_PATH + "cleaned_md_data_2.csv").drop(['Unnamed: 0'],axis=1)
df

,cell_type,source,title
0,markdown,House Prices: Advanced Regression Techniques P...,1-house-prices-solution-top-1.ipynb
1,markdown,Load packages,1-house-prices-solution-top-1.ipynb
2,commented code,# This Python 3 environment comes with many he...,1-house-prices-solution-top-1.ipynb
3,markdown,Load data,1-house-prices-solution-top-1.ipynb
4,code,train = pd.read_csv('../input/house-prices-adv...,1-house-prices-solution-top-1.ipynb
...,...,...,...
13368,code,submission_df,your-first-nlp-competition-submission.ipynb
13369,code,submission_df['target'].value_counts(),your-first-nlp-competition-submission.ipynb
13370,code,submission = submission_df.to_csv('Result.csv'...,your-first-nlp-competition-submission.ipynb
13371,markdown,"if you liked this kernel,please upvote it 3",your-first-nlp-competition-submission.ipynb


Let's see the commented codes

In [5]:
df[df["cell_type"]=="commented code"]

,cell_type,source,title
2,commented code,# This Python 3 environment comes with many he...,1-house-prices-solution-top-1.ipynb
21,commented code,"def spearman(frame, features):\r\n spr = pd...",1-house-prices-solution-top-1.ipynb
64,commented code,from datetime import datetime\r\n\r\nfrom scip...,1-house-prices-solution-top-1.ipynb
65,commented code,# Based on https://www.kaggle.com/hemingwei/to...,1-house-prices-solution-top-1.ipynb
67,commented code,"print('Predict submission', datetime.now(),)\r...",1-house-prices-solution-top-1.ipynb
...,...,...,...
13326,commented code,"# consider both unigrams and bigrams, occur at...",your-first-nlp-competition-submission.ipynb
13334,commented code,# Convert a collection of text documents to a ...,your-first-nlp-competition-submission.ipynb
13340,commented code,"word = ""forest""\r\ndic_vocabulary[word]\r\n#If...",your-first-nlp-competition-submission.ipynb
13359,commented code,#from sklearn.neighbors import KNeighborsClass...,your-first-nlp-competition-submission.ipynb


Function that extract commments and code :

In [6]:
def extract_comments(new_df,code_cell,title):

    # spliting code_cell into lines
    List = code_cell.splitlines()
    for i in List :
         
        # Ignore elements that contain only line break 
        if str(i) != '':

            #The tokenize() generator requires one argument: readline ,hence the string's conversion
            stringio = StringIO(i)
            res = ""
            space =""
            
            for toktype, tokval, begin, end, line in tokenize.generate_tokens(stringio.readline):
                        
                        if toktype != tokenize.COMMENT:
                            res+=tokval+" "
                            space +=" "
                        else :
                            new_df.loc[len(new_df)] = ['markdown',tokenize.untokenize([(toktype, tokval)]),title]

                    
            if res != space:
                        new_df.loc[len(new_df)] = ['code',res,title]
    


Function that goes through all the rows of the dataframe and fills a new dataframe while extracting the comments from code cells without affecting the order of the base cells

In [7]:
def build_df(new_df,old_df):
    c=0
    for i in old_df.index:
        if old_df.loc[i]['cell_type'] == "commented code":
                try:
                    extract_comments(new_df,str(old_df.loc[i]['source'])+"\n",old_df.loc[i]['title'])     
                except Exception as e :
                    c+=1
        else:
            new_df.loc[len(new_df)] = old_df.loc[i]
    print(c)

In [8]:
build_df(new_df,df)

679


In [9]:
new_df

,cell_type,source,title
0,markdown,House Prices: Advanced Regression Techniques P...,1-house-prices-solution-top-1.ipynb
1,markdown,Load packages,1-house-prices-solution-top-1.ipynb
2,markdown,# This Python 3 environment comes with many he...,1-house-prices-solution-top-1.ipynb
3,markdown,# It is defined by the kaggle/python docker im...,1-house-prices-solution-top-1.ipynb
4,markdown,"# For example, here's several helpful packages...",1-house-prices-solution-top-1.ipynb
...,...,...,...
36120,code,submission_df,your-first-nlp-competition-submission.ipynb
36121,code,submission_df['target'].value_counts(),your-first-nlp-competition-submission.ipynb
36122,code,submission = submission_df.to_csv('Result.csv'...,your-first-nlp-competition-submission.ipynb
36123,markdown,"if you liked this kernel,please upvote it 3",your-first-nlp-competition-submission.ipynb


Reclean markdowns ( Extracted comments)

In [10]:
def clean_md(raw_df,df):
    for i in raw_df.index:
        if raw_df.loc[i]['cell_type'] == 'markdown':

                string = EMOJI_PATTERN.sub(r'',re.sub(' +', ' ',(re.sub(r'(?:_|[^\s\w])(?!(?<=\.))(?!(?<=\:))(?!(?<=\,))(?!(?<=\?))(?!(?<=\!))', ' ',re.sub(r"\n",' ',re.sub(r'(\s)@\w+', r'\1',RMV_URL.sub(r'',BeautifulSoup(mdtex2html.convert(re.sub("\$","",re.sub(RMV_EQU," ",str(raw_df.loc[i]['source']), 0, re.MULTILINE))), "lxml").text)))))))
                df.loc[len(df)]=[raw_df.loc[i]['cell_type'],string,raw_df.loc[i]['title']]

        else :
            df.loc[len(df)]=raw_df.loc[i]

In [11]:
cleaned_df = pd.DataFrame(columns=['cell_type','source','title'])
clean_md(new_df,cleaned_df)
cleaned_df

,cell_type,source,title
0,markdown,House Prices: Advanced Regression Techniques P...,1-house-prices-solution-top-1.ipynb
1,markdown,Load packages,1-house-prices-solution-top-1.ipynb
2,markdown,This Python 3 environment comes with many help...,1-house-prices-solution-top-1.ipynb
3,markdown,It is defined by the kaggle python docker image:,1-house-prices-solution-top-1.ipynb
4,markdown,"For example, here s several helpful packages t...",1-house-prices-solution-top-1.ipynb
...,...,...,...
36120,code,submission_df,your-first-nlp-competition-submission.ipynb
36121,code,submission_df['target'].value_counts(),your-first-nlp-competition-submission.ipynb
36122,code,submission = submission_df.to_csv('Result.csv'...,your-first-nlp-competition-submission.ipynb
36123,markdown,"if you liked this kernel,please upvote it 3",your-first-nlp-competition-submission.ipynb


In [12]:
cleaned_df.to_csv(DATA_PATH+"data_comment_treated.csv")